<a href="https://colab.research.google.com/github/shacharki/AweSimonSays/blob/main/RDD_Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a3a22d81f9dceeb9f2c7facdba1e38ba368d5b022d63dac26fb1a0060a2c9619
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

In [3]:
#פונקציה שלוקחת מהקובץ וממפה את הסרט והשם השם שלו בתוך מילון
#קי היד של הסרטים ווליו שם הסרט
def load_movie_names():
  movie_names = {}
  with open("u.item",encoding='ISO-8859-1') as file:
    for line in file:
      fields = line.split('|')
      movie_names[int(fields[0])] = fields[1]
  return movie_names

In [4]:
def parse_input(line):
  fields = line.split()
  #המרת כל שורה לטאפל של 2 ערכים
  return (int(fields[1]),(int(fields[2]),1))

In [10]:
#למצוא את הסרטים עם הדירוג הכי נמוך והכי גבוהה
#ושיש להם לפחות 10 דירוגים
if __name__ == "__main__":
  spark = SparkSession.builder.appName("MovieLens").getOrCreate()
  #יצירת מילון ביחס לשם והיד שלו
  movie_names = load_movie_names()
  #קבלת הדאטה של הרדד
  lines = spark.sparkContext.textFile("u.data")
  print(lines.take(10))
  #מיפוי כל שורה
  #הפיכת הרדד למבנה של קי ווליו
  movies = lines.map(parse_input)
  print(movies.take(10))
  #רדד של קי ווליו שנוכל להפעיל מתודות שלא ניתן לעשות על רדד רגיל
  #רדיוס בי קי לפי היד של הסרט
  #סכימת הדירוג לפי כמות הצבעות
  ratingTotal = movies.reduceByKey(lambda movie1,movie2: (movie1[0]+movie2[0], movie1[1] + movie2[1]))
  #[יד של הסרט, (כמות מצטברת של דירוג ההצבעות, כמות אנשים שהצביעו)]
  print(ratingTotal.take(10))
  #ניקח רק את הסרטים שיש להם מעל 10 דירוגים
  real_movies = ratingTotal.filter(lambda movie: movie[1][1] > 10)
  #כדי לעשות ממוצע נעשה כמות מצטברת של הציון חלקי כמות הצבעות
  averagerating = real_movies.mapValues(lambda movie: movie[0]/movie[1])
  print(averagerating.take(10))
  #הגענו לממוצע מצד ימין ניקח את ה10 הנמוכים ביותר

  sorted_movies = averagerating.sortBy(lambda movie: movie[1], ascending=False)
  #קיבלנו יד של סרט וממצוע של הדירוגים
  #
  print(sorted_movies.take(10))
  for res in sorted_movies.take(10):
    print(movie_names[res[0]], res[1])

  #מס מילים בקובץ נשתמש ברדד רגיל ומשהו מסובך כמו פה נעדיף דאטה פריים של רדד
  #אם נעבוד עם דאטה של טבלה של נתונים נעדיף דאטה פריים
  #קובץ כגון ספר שנרצה לספור מילים אז רדד רגיל

['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116', '244\t51\t2\t880606923', '166\t346\t1\t886397596', '298\t474\t4\t884182806', '115\t265\t2\t881171488', '253\t465\t5\t891628467', '305\t451\t3\t886324817', '6\t86\t3\t883603013']
[(242, (3, 1)), (302, (3, 1)), (377, (1, 1)), (51, (2, 1)), (346, (1, 1)), (474, (4, 1)), (265, (2, 1)), (465, (5, 1)), (451, (3, 1)), (86, (3, 1))]
[(242, (467, 117)), (302, (1236, 297)), (346, (459, 126)), (474, (825, 194)), (86, (591, 150)), (1014, (300, 98)), (222, (1336, 365)), (40, (165, 57)), (274, (665, 190)), (1042, (88, 28))]
[(242, 3.9914529914529915), (302, 4.161616161616162), (346, 3.642857142857143), (474, 4.252577319587629), (86, 3.94), (1014, 3.061224489795918), (222, 3.66027397260274), (40, 2.8947368421052633), (274, 3.5), (1042, 3.142857142857143)]
[(408, 4.491071428571429), (318, 4.466442953020135), (169, 4.466101694915254), (483, 4.45679012345679), (114, 4.447761194029851), (64, 4.445229681978798), (603, 4.38755980